In [ ]:
# Install all necessary libraries
!pip install -q -U langgraph langchain langchain-google-genai "unstructured[pdf]" "deeplake[enterprise]" faiss-cpu tavily-python python-dotenv langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 31.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 89.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 106.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 110.2 MB/s eta 0:00:00
   ━━━━━━━

In [ ]:
import os
from google.colab import userdata

# Load API keys from Colab secrets
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')

print("API Keys loaded successfully.")

API Keys loaded successfully.


In [ ]:
# Download some sample financial reports
!wget -q -O tsla_10k_2023.pdf "https://www.sec.gov/Archives/edgar/data/1318605/000162828024002390/tsla-20231231.htm"
!wget -q -O tsla_q1_2024_update.pdf "https://digitalassets.tesla.com/tesla-contents/image/upload/IR/TSLA-Q1-2024-Update.pdf"

from langchain_community.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# List of your document paths
doc_paths = ["tsla_10k_2023.pdf", "tsla_q1_2024_update.pdf"]
all_docs = []
for path in doc_paths:
    try:
        # Using UnstructuredFileLoader is more robust
        loader = UnstructuredFileLoader(path)
        all_docs.extend(loader.load())
        print(f"Successfully loaded {path}")
    except Exception as e:
        print(f"Could not load {path}. Error: {e}")

# Split documents into manageable chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)
chunks = text_splitter.split_documents(all_docs)

print(f"\nCreated {len(chunks)} document chunks from the corpus.")

<ipython-input-7-9b1748ba3489>:14: LangChainDeprecationWarning: The class `UnstructuredFileLoader` was deprecated in LangChain 0.2.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-unstructured package and should be used instead. To use it run `pip install -U :class:`~langchain-unstructured` and import as `from :class:`~langchain_unstructured import UnstructuredLoader``.
  loader = UnstructuredFileLoader(path)


Successfully loaded tsla_10k_2023.pdf


Successfully loaded tsla_q1_2024_update.pdf

Created 32 document chunks from the corpus.


In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.tools.retriever import create_retriever_tool

# Use Google's embedding model for creating vectors
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Create the FAISS vector store in memory from our document chunks
vectorstore = FAISS.from_documents(chunks, embeddings)

# Create a retriever which will find the relevant chunks
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# Create the final tool for the agent
document_retriever_tool = create_retriever_tool(
    retriever,
    "document_search",
    "Searches the provided financial documents (10-K reports, quarterly updates) to answer questions about Tesla's historical performance, strategy, risks, and financials."
)

print("RAG retriever tool ('document_search') created successfully.")

RAG retriever tool ('document_search') created successfully.


In [ ]:
from langchain_core.tools import tool
from tavily import TavilyClient
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

# --- Define Web Search Tool using Tavily ---
@tool
def web_search_tool(query: str):
    """Searches the web for real-time information like stock prices and breaking news."""
    tavily_client = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])
    # Using 'search' which is comprehensive.
    response = tavily_client.search(query, search_depth="advanced")
    # Return a formatted string of the most relevant results
    return "\n".join([f"- {c['content']}" for c in response['results']])

# --- Define the LLM for all agents ---
# Using gemini-1.5-pro for its large context window and strong reasoning
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest", temperature=0, convert_system_message_to_human=True)

# --- Agent Factory Function (to create agent executors) ---
def create_agent(llm, tools, system_prompt_str):
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt_str),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ])
    agent = create_tool_calling_agent(llm, tools, prompt)
    return AgentExecutor(agent=agent, tools=tools, verbose=True)

# --- Agent 1: Document Research Agent ---
document_agent_executor = create_agent(
    llm,
    [document_retriever_tool], # Its only tool is the RAG retriever
    "You are a meticulous financial analyst specializing in dissecting internal company documents. Use your 'document_search' tool to find specific information within the provided reports."
)

# --- Agent 2: Web Research Agent ---
web_agent_executor = create_agent(
    llm,
    [web_search_tool], # Its only tool is the web search
    "You are a real-time market analyst. You must use your web search tool to find the most current information, including stock prices and breaking news."
)

print("Specialist agents (Document Analyst and Web Analyst) are ready.")

Specialist agents (Document Analyst and Web Analyst) are ready.


In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import SystemMessage, HumanMessage

# --- Define the State for the Graph ---
class AgentState(TypedDict):
    task: str
    plan: str
    # Each list item is a tuple of (agent_name, agent_output)
    history: Annotated[list, operator.add]
    report: str

# --- Create the Supervisor Chain ---
# We will construct messages explicitly instead of using a prompt template directly in the chain
supervisor_prompt_str = """You are the supervisor of a team of expert AI agents. Your job is to create a final, comprehensive report based on the user's request and the findings from your agents.

User's Original Request: {task}

Your Plan:
{plan}

Agent Findings:
{history}

Based on all of this information, synthesize a single, cohesive, and well-written final report. The report should directly answer the user's original request.
"""
# Removed the chain creation here: supervisor_chain = supervisor_prompt | llm


# --- Simplified Sequential Execution Logic ---
def run_graph(task: str):
    # For this example, we define a simple, static plan.
    plan = """1. Use the Document_Analyst to check the latest quarterly and annual reports for official statements on vehicle production, financial health, and stated risks.
2. Use the Web_Analyst to get the absolute latest stock price and any breaking news from the last 48 hours that might not be in the reports."""

    print(f"Supervisor's Plan:\n{plan}\n" + "="*50)

    # Step 1: Run the document agent
    print("\nDelegating to Document Analyst...")
    doc_agent_input = {"input": f"Based on the user's task: '{task}', execute step 1 of the plan."}
    doc_result = document_agent_executor.invoke(doc_agent_input)
    history = [("Document_Analyst", doc_result['output'])]
    print("="*50)

    # Step 2: Run the web agent
    print("\nDelegating to Web Analyst...")
    web_agent_input = {"input": f"Based on the user's task: '{task}', execute step 2 of the plan."}
    web_result = web_agent_executor.invoke(web_agent_input)
    history.append(("Web_Analyst", web_result['output']))
    print("="*50)

    # Format history for the supervisor
    formatted_history = "\n".join([f"Result from {agent}:\n{result}\n" for agent, result in history])

    # Step 3: Supervisor synthesizes the final report
    print("\nSupervisor is synthesizing the final report...")

    # Construct messages explicitly
    messages = [
        SystemMessage(content="You are the supervisor of a team of expert AI agents. Your job is to create a final, comprehensive report based on the user's request and the findings from your agents."),
        HumanMessage(content=f"User's Original Request: {task}\n\nYour Plan:\n{plan}\n\nAgent Findings:\n{formatted_history}\n\nBased on all of this information, synthesize a single, cohesive, and well-written final report. The report should directly answer the user's original request.")
    ]

    final_report = llm.invoke(messages) # Invoke the LLM directly with the messages

    print("\n\n✅ FINAL COMPREHENSIVE REPORT:\n")
    print(final_report.content)

# --- Execute the entire process ---
user_task = "Provide a summary of Tesla's Q1 2024 performance, key risks mentioned in their reports, and their current stock price."
run_graph(user_task)

Supervisor's Plan:
1. Use the Document_Analyst to check the latest quarterly and annual reports for official statements on vehicle production, financial health, and stated risks.
2. Use the Web_Analyst to get the absolute latest stock price and any breaking news from the last 48 hours that might not be in the reports.

Delegating to Document Analyst...


> Entering new AgentExecutor chain...


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")



Invoking: `document_search` with `{'query': 'Tesla Q1 2024 performance summary'}`


1

Q1 2024 Update

Highlights

Financial Summary

Operational Summary

Vehicle Capacity

Core Technology

Other Highlights

Outlook

Photos & Charts

Key Metrics

Financial Statements

Additional Information

03

04

06

07

08

09

10

11

22

24

30

H I G H L I G H T S

S U M M A R Y

Profitability

$1.2B GAAP operating income in Q1

$1.1B GAAP net income in Q1

We experienced numerous challenges in Q1, from the Red Sea conflict and the arson attack at Gigafactory Berlin, to the gradual ramp of the updated Model 3 in Fremont. Excluding Cybertruck and unscheduled downtime, our COGS4 per unit declined sequentially, driven primarily by lower raw material costs.

$1.5B non-GAAP net income1 in Q1

Global EV sales continue to be under pressure as many carmakers prioritize hybrids over EVs. While positive for our regulatory credits business, we prefer the industry to continue pushing EV adoption, which is 

/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Based on the document search, Tesla's Q1 2024 performance shows a GAAP operating income of $1.2B and a GAAP net income of $1.1B.  Non-GAAP net income was $1.5B.  Operating cash flow was $0.2B, but free cash flow was negative $2.5B due to significant capital expenditures ($2.8B in Q1, including $1.0B for AI infrastructure).  Vehicle production was impacted by challenges including the Red Sea conflict, an arson attack at Gigafactory Berlin, and the ramp-up of the updated Model 3 in Fremont.  Despite these challenges, COGS per unit declined sequentially due to lower raw material costs (excluding Cybertruck and unscheduled downtime).  The company is investing heavily in future growth, including AI infrastructure and new product development.  More details on specific metrics and operational summaries are available in the full report.  The provided text does not contain Tesla's current stock price.  Key risks mentioned include the global pressure on EV sales, the Red Sea conflict, and the ar

...


Invoking: `web_search_tool` with `{'query': 'Tesla Q1 2024 performance, key risks, current stock price'}`


- On April 23, 2024, Tesla Inc ([NASDAQ:TSLA](https://www.gurufocus.com/term/gf-score/TSLA)) released its [8-K filing](https://www.sec.gov//Archives/edgar/data/1318605/000095017024046895/tsla-ex99_1.htm), detailing the financial outcomes for the first quarter of 2024. The company reported a significant downturn in key financial metrics, notably underperforming against analyst expectations for both earnings per share and revenue. Tesla's reported earnings per share (EPS) of $0.34 sharply [...] **Operating Cash Flow:** Plummeted by 90% year-over-year to $242 million, indicating a sharp decline in operational efficiency.

**Gross Margin:** Decreased by 199 basis points to 17.4%, impacted by reduced vehicle average selling prices and increased operating expenses.

**Capital Expenditures:** Rose by 34% to $2.77 billion, underscoring continued investment in growth despite financial pr

/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Tesla's Q1 2024 performance showed a significant downturn compared to analyst expectations and the previous year.  Key financial metrics like EPS and revenue underperformed.  Operating cash flow plummeted by 90% year-over-year, and gross margin decreased.  Net income was down 55% year-over-year to $1.1 billion, and free cash flow was negative at $2.5 billion.  The company cited increased operational costs and lower vehicle sales as contributing factors.  Tesla's outlook for 2024 is pessimistic, with projected volume growth significantly lower than in 2023.  Key risks mentioned include disappointing Q1 2024 deliveries, falling short of analyst expectations, concerns about production issues and future demand, and increased competition, particularly in China.  The current stock price information was not directly provided in the search results but can be easily found through a financial website.  The provided text mentions that prior to a post-earnings jump, Tesla shares were down more tha

/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")




✅ FINAL COMPREHENSIVE REPORT:

**Tesla Q1 2024 Performance Report**

This report summarizes Tesla's Q1 2024 performance, key risks identified, and its current stock price.

**Financial Performance:**

Tesla reported a GAAP net income of $1.1 billion and a GAAP operating income of $1.2 billion for Q1 2024.  Non-GAAP net income reached $1.5 billion. While these figures represent profitability, they represent a significant downturn compared to the previous year and analyst expectations.  Operating cash flow was a meager $0.2 billion, while free cash flow was negative $2.5 billion, primarily due to substantial capital expenditures ($2.8 billion, including $1 billion for AI infrastructure).  This negative free cash flow and the significant year-over-year decrease in operating cash flow (reported as a 90% decline by Web_Analyst) are significant concerns.  The company attributed the lower-than-expected performance to increased operational costs and lower vehicle sales.  Cost of Goods Sold (

In [ ]:
# Cell 1: Install Dependencies
!pip install -q -U langgraph langchain langchain-google-genai "unstructured[pdf]" faiss-cpu tavily-python python-dotenv pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 86.0 MB/s eta 0:00:00
